In [2]:
#加载需要的库
import time
import math
import datetime
import numpy as np
import pandas as pd

import tushare as ts
import backtrader as bt
from sklearn.svm import SVR

from ast import literal_eval
import matplotlib.pyplot as plt
from backtrader.feeds import PandasData
from sklearn.preprocessing import RobustScaler#%%


In [3]:
#实例化支持向量回归（SVR）模型
svr = SVR()

In [4]:
from mfm_learner.utils import CONF
from mfm_learner.datasource import datasource_factory

datasource = datasource_factory.create(CONF['datasource'])
index_code = '000001.SH'
start_date = '2019-01-01'
end_date = '2020-01-01'
stock_codes = datasource.index_weight(index_code, start_date, end_date)
stock_codes = stock_codes[:100]
df_daily = datasource.daily_basic(stock_codes, start_date, end_date)
df_daily2 = datasource.daily(stock_codes, start_date, end_date)

/Users/wangtianshu/PycharmProjects/mfm_learner/mfm_learner/datasource/impl/database_datasource.py:71: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  return pd.concat(df_basics)


In [5]:
# inner join 2 dataframes by code-datatime tuple, for same name columns, just keep the first one
merged_df = pd.merge(df_daily, df_daily2, on=['code', 'datetime'], suffixes=('', '_drop'))
merged_df

,code,datetime,close,turnover_rate,turnover_rate_f,volume_ratio,pe,pe_ttm,pb,ps,...,circ_mv,open,high,low,close_drop,pre_close,change,pct_chg,volume,amount
0,601398.SH,20190102,5.20,0.0420,0.5249,0.98,6.4790,6.2259,0.8498,2.5510,...,1.401984e+08,5.30,5.31,5.17,5.20,5.29,-0.09,-1.70,1131849.00,589032.055
1,601398.SH,20190103,5.20,0.0350,0.4374,0.81,6.4790,6.2259,0.8498,2.5510,...,1.401984e+08,5.18,5.23,5.17,5.20,5.20,0.00,0.00,943284.27,489960.755
2,601398.SH,20190104,5.27,0.0454,0.5678,1.20,6.5662,6.3097,0.8612,2.5853,...,1.420856e+08,5.18,5.28,5.15,5.27,5.20,0.07,1.35,1224420.27,639955.298
3,601398.SH,20190107,5.24,0.0371,0.4636,0.92,6.5288,6.2738,0.8563,2.5706,...,1.412768e+08,5.30,5.30,5.23,5.24,5.27,-0.03,-0.57,999757.87,525367.219
4,601398.SH,20190108,5.20,0.0419,0.5236,1.04,6.4790,6.2259,0.8498,2.5510,...,1.401984e+08,5.25,5.25,5.18,5.20,5.24,-0.04,-0.76,1129184.31,586983.187
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24124,600674.SH,20191225,9.70,0.0893,0.2855,0.49,11.9608,12.4514,1.6334,49.4478,...,4.270076e+06,9.71,9.73,9.65,9.70,9.72,-0.02,-0.21,39302.49,38035.931
24125,600674.SH,20191226,9.71,0.1211,0.3872,0.73,11.9732,12.4642,1.6351,49.4988,...,4.274478e+06,9.71,9.73,9.63,9.71,9.70,0.01,0.10,53307.92,51537.983
24126,600674.SH,20191227,9.70,0.1388,0.4439,0.91,11.9608,12.4514,1.6334,49.4478,...,4.270076e+06,9.70,9.76,9.66,9.70,9.71,-0.01,-0.10,61107.83,59339.920
24127,600674.SH,20191230,9.78,0.1937,0.6194,1.36,12.0595,12.5541,1.6469,49.8556,...,4.305293e+06,9.70,9.80,9.61,9.78,9.70,0.08,0.82,85276.22,83162.884


In [6]:
def get_stock_data(code):
    df_stock = merged_df[merged_df['code']==code]
    df_stock = df_stock[['datetime','open','high','low','close','volume','turnover_rate_f','circ_mv','total_mv']]
    df_stock['datetime'] = pd.to_datetime(df_stock['datetime'])
    df_stock.index = df_stock.datetime
    df_stock = df_stock.sort_index()
    return df_stock

In [7]:
# make a df to get available stock list by day
list_res = []
for date in merged_df.datetime.unique():
    stocklist = merged_df[merged_df['datetime']==date]['code'].tolist()
    list_res.append({'Date':date,'stocklist':stocklist})
df_stocklist = pd.DataFrame(list_res)
df_stocklist['Date'] = pd.to_datetime(df_stocklist['Date'])

In [8]:
class AddMoreColumns(PandasData):
    '''
    Add more data to the inherited PandasData
    '''
    # Add the parameter to the inherited ones
    lines = ('turnover_rate_f','circ_mv','total_mv',)
    params = (('turnover_rate_f',-1),('circ_mv',-1),('total_mv',-1),)

In [25]:
#设置佣金和印花税率
class stampDutyCommissionScheme(bt.CommInfoBase):
    '''
    本佣金模式下，买入股票仅支付佣金，卖出股票支付佣金和印花税.    
    '''
    params = (
        ('stamp_duty', 0.001), # 印花税率
        ('commission', 0.0005), # 佣金率
        ('stocklike', True),
        ('commtype', bt.CommInfoBase.COMM_PERC),
        )
 
    def _getcommission(self, size, price, pseudoexec):
        '''
        If size is greater than 0, this indicates a long / buying of shares.
        If size is less than 0, it idicates a short / selling of shares.
        '''
        #print('self.p.commission',self.p.commission)
        if size > 0: # 买入，不考虑印花税
            return  size * price * self.p.commission * 100
        elif size < 0: # 卖出，考虑印花税
            return - size * price * (self.p.stamp_duty + self.p.commission*100)
        else:
            return 0 

In [32]:
#编写策略
class momentum_factor_strategy(bt.Strategy):
    #interval-换仓间隔，stocknum-持仓股票数
    params = (("interval",1),("stocknum",10),)

    def log(self, txt, dt=None):
        ''' Logging function fot this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('{}, {}'.format(dt.isoformat(), txt))

    def __init__(self):
        #已清洗过的每日可用股票列表
        self.df_byday = df_stocklist
        #交易天数，用于判断是否交易
        self.bar_num = 0
        #上次交易股票的列表
        self.last = []  

        # 记录以往订单，在调仓日要全部取消未成交的订单
        self.order_list = []

    def prenext(self):
        
        self.next()
        
    def next(self):
        #记录交易日期
        self.bar_num+=1
        print("当天日期:{}".format(str(self.datas[0].datetime.date(0))))
        #计算当日是否调仓
        if self.bar_num%self.p.interval==0 and self.bar_num > 3 * self.p.interval and self.datas[0].datetime.date(0) < datetime.date(2020,6,25):
            #得到当天的时间
            current_date=self.datas[0].datetime.date(0)
            print("交易日日期:{}".format(str(self.datas[0].datetime.date(0))))
            #获得上一调仓日时间
            prev_date=self.datas[0].datetime.date(-self.p.interval)
            #获取当日可行股票池
            stocklist = self.get_valid_list_day(current_date)
            #获取上一调仓日可行股票池
            stocklist_p = self.get_valid_list_day(prev_date)

            #计算本期因子数据df_fac，并清洗
            df_fac = self.get_df_fac(stocklist=stocklist,prev=0)
            df_fac = df_fac.dropna(axis=0,how='any')

            #计算上期因子数据df_faxc_p，并清洗
            df_fac_p = self.get_df_fac(stocklist=stocklist_p,prev=1)
            df_fac_p = df_fac_p.dropna(axis=0,how='any')

            #本期因子排列命名
            df_fac.columns = ['code','momentum_value','turnover_rate_f','circ_mv','total_mv']
            df_fac.index = df_fac.code.values

            #上期因子排列命名
            df_fac_p.columns = ['code','momentum_value','turnover_rate_f','circ_mv','total_mv']
            df_fac_p.index = df_fac_p.code.values

            #舍弃X_p和Y中不同的index（股票代码）
            #先去除X_p比Y多的index
            diffIndex = df_fac_p.index.difference(df_fac.index)
            #删除整行
            df_fac_p = df_fac_p.drop(diffIndex,errors='ignore')
            df_fac = df_fac.drop(diffIndex,errors='ignore')

            # 然后去除Y比X_p多的index
            diffIndex = df_fac.index.difference(df_fac_p.index)
            df_fac_p = df_fac_p.drop(diffIndex,errors='ignore')
            df_fac = df_fac.drop(diffIndex,errors='ignore')

            #X_p是上一期的因子值，X是本期因子值，Y是回归目标
            X_p = df_fac_p[['momentum_value','turnover_rate_f','circ_mv']]
            X = df_fac[['momentum_value','turnover_rate_f','circ_mv']]
            Y = df_fac[['momentum_value']]

            #将因子值与Y值均进行标准化
            rbX = RobustScaler()
            X_p = rbX.fit_transform(X_p)

            rbY = RobustScaler()
            Y = rbY.fit_transform(Y)
            
            #用上期因子值与本期回报率进行训练
            svr.fit(X_p,np.ravel(Y)) 
            
            #用本期因子值预测下期回报率
            svr_pred = svr.predict(rbX.transform(X))
            
            a = rbY.inverse_transform(svr_pred.reshape(-1, 1))
            df_fac['pred'] = a
            
            #按照预测得到的下期收益进行排序
            df_fac.sort_values(by="pred" , inplace=True, ascending=False)
            #取预测收益>0且排序靠前的stocknum只股票做多
            df_fac_pos = df_fac[df_fac['pred']>0]
            sort_list_pos = df_fac['code'].tolist()
            long_list=sort_list_pos[:self.p.stocknum]
            
            #取预测收益<0且排序靠后的stocknum只股票做空
            df_fac_neg = df_fac[df_fac['pred']<0]
            sort_list_neg = df_fac_neg['code'].tolist()
            short_list=sort_list_neg[-self.p.stocknum:]            

            #取消以往所下订单（已成交的不会起作用）
            for o in self.order_list:
                self.cancel(o)
            #重置订单列表
            self.order_list = []  
            
            #若上期交易股票未出现在本期交易列表中，则平仓
            for i in self.last:
                if i not in long_list and i not in short_list:
                    d = self.getdatabyname(i)
                    print('sell 平仓', d._name, self.getposition(d).size)
                    o = self.close(data=d)
                    self.order_list.append(o) # 记录订单
            
            self.log('当前总市值 %.2f' %(self.broker.getvalue()))
            
            #获取当前账户价值
            total_value = self.broker.getvalue()
            
            #对long_list中股票做多
            if len(long_list):
        
                #每只股票买入资金百分比，预留5%的资金以应付佣金和计算误差
                buypercentage = (1-0.05)/2/len(long_list)
    
                #得到目标市值
                targetvalue = buypercentage * total_value
                
                #依次买入
                for d in long_list:
                    data = self.getdatabyname(d)
                    #按次日开盘价计算下单量，下单量是100的整数倍
                    size = int(abs(targetvalue / data.open[1] // 100 * 100))
                    o = self.order_target_size(data=d, target=size)
                    #记录订单             
                    self.order_list.append(o) 

            #对short_list中股票做空                    
            if len(short_list):
                
                #每只股票做空资金百分比，预留5%的资金以应付佣金和计算误差
                buypercentage = (1-0.05)/2/len(short_list)
    
                #得到目标市值
                targetvalue = buypercentage * total_value
                #依次卖空
                for d in short_list:
                    data = self.getdatabyname(d)
                    #按次日开盘价计算下单量，下单量是100的整数倍
                    size = int(abs(targetvalue / data.open[1] // 100 * 100))
                    o = self.order_target_size(data=d, target=-size)
                    #记录订单             
                    self.order_list.append(o) 
            
            #跟踪上次交易的标的
            self.last = list(set(long_list).union(set(short_list)))  
            
    #交易日志    
    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enougth cash
        if order.status in [order.Completed, order.Canceled, order.Margin]:
            if order.isbuy():
                self.log(
                    'BUY EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f, Size: %.2f, Stock: %s' %
                    (order.executed.price,
                     order.executed.value,
                     order.executed.comm,
                     order.executed.size,
                     order.data._name))
            else:  # Sell
                self.log('SELL EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f, Size: %.2f, Stock: %s' %
                        (order.executed.price,
                         order.executed.value,
                         order.executed.comm,
                         order.executed.size,
                         order.data._name))
                

    def notify_trade(self, trade):
        if trade.isclosed:
            
            self.log('TRADE PROFIT, GROSS %.2f, NET %.2f' %
                     (trade.pnl, trade.pnlcomm))

    
    # 求个股某周期因子平均值，prev为是否为前一周期（0：否，1：是）
    def get_df_fac(self,stocklist,prev=0):
        #初始化若干个list，用于计算因子值
        #N-个股情绪因子，H-个股热度因子，TR_F-流通股换手率，CM-流通市值，TM-总市值（未使用）
        TR_F,CM,TM = [],[],[]
        #新建df_fac用于存储计算得到的因子值
        df_fac = pd.DataFrame(columns=['code','momentum_value','turnover_rate_f','circ_mv','total_mv']) 
        for stock in stocklist:
            data=self.getdatabyname(stock)
            
            #获取当期因子值的平均数
            for i in range(self.p.interval):
                # if data.news[(-i-prev*self.p.interval)] != 0:
                #     N.append(data.news[(-i-prev*self.p.interval)])

                # if data.heat[(-i-prev*self.p.interval)] != 0:
                #     H.append(data.heat[(-i-prev*self.p.interval)])
                TR_F.append(data.turnover_rate_f[(-i-prev*self.p.interval)])
                CM.append(data.circ_mv[(-i-prev*self.p.interval)])
                TM.append(data.total_mv[(-i-prev*self.p.interval)])
            
            #若当期无舆情因子数据，则不返回
            if 1:
                #缺失数据处理--若后续数据缺失，跳过该股票
                try:
                    data.open[1]
                except IndexError:
                    print('%s后续数据缺失，跳过该股票'%(stock))
                else:
                    if data.close[0] != 0 and data.open[1] !=0:
                        #舆情因子取最近一天的值与当期平均值之和（加大最近一天的权重）
                        # N = np.mean(N) + data.news[(-prev*self.p.interval)]
                        # H = np.mean(H) + data.heat[(-prev*self.p.interval)]
                        pass
                    
                    try:
                        data.open[1+self.p.interval-prev*self.p.interval]
                    except IndexError:
                        print('%s后续数据缺失，跳过该股票'%(stock))
                    else:
    
                        #计算当期动量
                        sell_ = data.open[1+self.p.interval-prev*self.p.interval]
                        buy_ = data.open[1-prev*self.p.interval]
                        stock_momentum = sell_ / buy_ - 1
                        #其余因子取当期平均值
                        TR_F = np.mean(TR_F)
                        CM = np.mean(CM)
                        TM = np.mean(TM)
    
                        new = pd.DataFrame({'code':stock,
                                            # 'news':N,
                                            # 'heat':H,
                                            'momentum_value':stock_momentum,
                                            'turnover_rate_f':TR_F,'circ_mv':CM,'total_mv':TM},index=[1]) 
                        # df_fac = df_fac.append(new,ignore_index=True)
                        df_fac = pd.concat([df_fac,new],axis=0)
            TR_F,CM,TM = [],[],[]
        return df_fac
    
    #获取当日可行股票池
    def get_valid_list_day(self,current_date):
        # self.df_byday['Date'] = pd.to_datetime(self.df_byday['Date'])
        current_date = datetime.datetime.strptime(str(current_date),'%Y-%m-%d')
        df_day = self.df_byday[self.df_byday['Date']==current_date]
        # stocklist = literal_eval(df_day['stocklist'].tolist()[0])
        stocklist = df_day['stocklist'].tolist()[0]
        return stocklist

In [33]:
r = df_stocklist[df_stocklist['Date']==datetime.datetime(2019,1,3)]
r['stocklist'].tolist()[0]

['601398.SH',
 '601288.SH',
 '601857.SH',
 '600519.SH',
 '601988.SH',
 '601318.SH',
 '600036.SH',
 '600028.SH',
 '601628.SH',
 '600900.SH',
 '601166.SH',
 '601088.SH',
 '600000.SH',
 '600104.SH',
 '601668.SH',
 '601328.SH',
 '601138.SH',
 '600276.SH',
 '600016.SH',
 '601766.SH',
 '601319.SH',
 '603288.SH',
 '601998.SH',
 '601601.SH',
 '600050.SH',
 '601818.SH',
 '600019.SH',
 '600048.SH',
 '600887.SH',
 '601360.SH',
 '600585.SH',
 '601006.SH',
 '601169.SH',
 '601211.SH',
 '601229.SH',
 '601688.SH',
 '601800.SH',
 '601390.SH',
 '600018.SH',
 '600015.SH',
 '601186.SH',
 '601888.SH',
 '601989.SH',
 '600690.SH',
 '600009.SH',
 '600309.SH',
 '600406.SH',
 '601336.SH',
 '601225.SH',
 '601985.SH',
 '600999.SH',
 '600340.SH',
 '601111.SH',
 '600837.SH',
 '603259.SH',
 '600606.SH',
 '601933.SH',
 '601669.SH',
 '601066.SH',
 '601238.SH',
 '600919.SH',
 '600031.SH',
 '600011.SH',
 '603993.SH',
 '601939.SH',
 '600010.SH',
 '601155.SH',
 '600346.SH',
 '600741.SH',
 '601012.SH',
 '600029.SH',
 '6000

In [34]:
begin_time=time.time()
cerebro = bt.Cerebro(stdstats=False)
comminfo=stampDutyCommissionScheme(stamp_duty=0.001,commission=0.0005)
cerebro.broker.addcommissioninfo(comminfo)
for s in stock_codes:
    feed = AddMoreColumns(dataname = get_stock_data(s),plot=False,
                       fromdate=datetime.datetime(2018,1,3),todate=datetime.datetime(2020,1,1))
    cerebro.adddata(feed, name = s)

In [35]:
cerebro.broker.setcash(1000000.0)
#防止下单时现金不够被拒绝。只在执行时检查现金够不够。
cerebro.broker.set_checksubmit(False)
# 添加相应的费用，杠杆率
# 获取策略运行的指标
print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

startcash=1000000.0
cerebro.addstrategy(momentum_factor_strategy) 
cerebro.addobserver(bt.observers.Value)

#添加Analyzer
cerebro.addanalyzer(
    bt.analyzers.SharpeRatio,
    riskfreerate=0.01,
    annualize=True,
    _name='sharp_ratio')
cerebro.addanalyzer(bt.analyzers.DrawDown, _name='drawdown')


Starting Portfolio Value: 1000000.00


In [36]:
len(cerebro.datas)

100

In [37]:

thestrats = cerebro.run()
thestrat = thestrats[0]
# 输出分析器结果字典
print('Sharpe Ratio:', thestrat.analyzers.sharp_ratio.get_analysis())
print('DrawDown:', thestrat.analyzers.drawdown.get_analysis())

# 进一步从字典中取出需要的值
print('Sharpe Ratio:',
      thestrat.analyzers.sharp_ratio.get_analysis()['sharperatio'])
print('Max DrawDown:',
      thestrat.analyzers.drawdown.get_analysis()['max']['drawdown'])

# 打印各个分析器内容
for a in thestrat.analyzers:
    a.print()
cerebro.plot() 
#获取回测结束后的总资金
portvalue = cerebro.broker.getvalue()
pnl = portvalue - startcash
#打印结果
print(f'总资金: {round(portvalue,2)}')
print(f'净收益: {round(pnl,2)}')
end_time=time.time()
print("一共使用时间为:{}".format(end_time-begin_time))

当天日期:2019-01-02
当天日期:2019-01-03
当天日期:2019-01-04
当天日期:2019-01-07
交易日日期:2019-01-07
2019-01-07, 当前总市值 1000000.00
2019-01-08, BUY EXECUTED, Price: 5.72, Cost: 47476.00, Comm 23.74, Size: 8300.00, Stock: 601668.SH
2019-01-08, BUY EXECUTED, Price: 28.44, Cost: 45504.00, Comm 22.75, Size: 1600.00, Stock: 601601.SH
2019-01-08, BUY EXECUTED, Price: 5.18, Cost: 47138.00, Comm 23.57, Size: 9100.00, Stock: 600028.SH
2019-01-08, BUY EXECUTED, Price: 20.24, Cost: 46552.00, Comm 23.28, Size: 2300.00, Stock: 601628.SH
2019-01-08, BUY EXECUTED, Price: 24.59, Cost: 46721.00, Comm 23.36, Size: 1900.00, Stock: 600104.SH
2019-01-08, BUY EXECUTED, Price: 15.22, Cost: 47182.00, Comm 23.59, Size: 3100.00, Stock: 601166.SH
2019-01-08, BUY EXECUTED, Price: 4.60, Cost: 47380.00, Comm 23.69, Size: 10300.00, Stock: 601989.SH
2019-01-08, BUY EXECUTED, Price: 10.03, Cost: 47141.00, Comm 23.57, Size: 4700.00, Stock: 600000.SH
2019-01-08, BUY EXECUTED, Price: 12.18, Cost: 46284.00, Comm 23.14, Size: 3800.00, Stock: 60

/var/folders/6w/1xbgl6h554l61vy5w5s0zqlc0000gn/T/ipykernel_15494/2530719429.py:249: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_fac = pd.concat([df_fac,new],axis=0)
/var/folders/6w/1xbgl6h554l61vy5w5s0zqlc0000gn/T/ipykernel_15494/2530719429.py:249: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_fac = pd.concat([df_fac,new],axis=0)
/var/folders/6w/1xbgl6h554l61vy5w5s0zqlc0000gn/T/ipykernel_15494/2530719429.py:249: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA 

sell 平仓 601901.SH 6700
sell 平仓 600028.SH -10900
sell 平仓 601012.SH 2200
sell 平仓 601318.SH -1000
sell 平仓 600115.SH 9300
sell 平仓 601319.SH 8600
sell 平仓 601138.SH 3800
sell 平仓 600547.SH 1500
sell 平仓 600588.SH 1900
sell 平仓 600029.SH 6500
2019-01-11, 当前总市值 980081.31
2019-01-14, SELL EXECUTED, Price: 6.85, Cost: 46431.00, Comm 68.84, Size: -6700.00, Stock: 601901.SH
2019-01-14, BUY EXECUTED, Price: 5.32, Cost: -57879.00, Comm 28.99, Size: 10900.00, Stock: 600028.SH
2019-01-14, SELL EXECUTED, Price: 20.03, Cost: 45980.00, Comm 66.10, Size: -2200.00, Stock: 601012.SH
2019-01-14, BUY EXECUTED, Price: 57.74, Cost: -58000.00, Comm 28.87, Size: 1000.00, Stock: 601318.SH
2019-01-14, SELL EXECUTED, Price: 5.10, Cost: 46221.00, Comm 71.14, Size: -9300.00, Stock: 600115.SH
2019-01-14, SELL EXECUTED, Price: 5.51, Cost: 46182.00, Comm 71.08, Size: -8600.00, Stock: 601319.SH
2019-01-14, SELL EXECUTED, Price: 12.21, Cost: 45676.00, Comm 69.60, Size: -3800.00, Stock: 601138.SH
2019-01-14, SELL EXECUTED, Pri

/var/folders/6w/1xbgl6h554l61vy5w5s0zqlc0000gn/T/ipykernel_15494/2530719429.py:249: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_fac = pd.concat([df_fac,new],axis=0)
/var/folders/6w/1xbgl6h554l61vy5w5s0zqlc0000gn/T/ipykernel_15494/2530719429.py:249: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_fac = pd.concat([df_fac,new],axis=0)
/var/folders/6w/1xbgl6h554l61vy5w5s0zqlc0000gn/T/ipykernel_15494/2530719429.py:249: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA 

sell 平仓 601901.SH 7300
sell 平仓 601360.SH 2200
sell 平仓 601238.SH 4600
sell 平仓 600760.SH 1500
sell 平仓 601877.SH 1800
sell 平仓 601398.SH 9000
sell 平仓 601138.SH 4000
2019-01-18, 当前总市值 1005727.37
2019-01-21, SELL EXECUTED, Price: 6.48, Cost: 47523.00, Comm 70.96, Size: -7300.00, Stock: 601901.SH
2019-01-21, SELL EXECUTED, Price: 20.84, Cost: 45848.00, Comm 68.77, Size: -2200.00, Stock: 601360.SH
2019-01-21, SELL EXECUTED, Price: 10.28, Cost: 47288.00, Comm 70.93, Size: -4600.00, Stock: 601238.SH
2019-01-21, SELL EXECUTED, Price: 31.00, Cost: 46725.00, Comm 69.75, Size: -1500.00, Stock: 600760.SH
2019-01-21, SELL EXECUTED, Price: 25.88, Cost: 46152.00, Comm 69.88, Size: -1800.00, Stock: 601877.SH
2019-01-21, SELL EXECUTED, Price: 5.41, Cost: 47250.00, Comm 73.03, Size: -9000.00, Stock: 601398.SH
2019-01-21, SELL EXECUTED, Price: 12.02, Cost: 47640.00, Comm 72.12, Size: -4000.00, Stock: 601138.SH
2019-01-21, SELL EXECUTED, Price: 7.26, Cost: 1426.00, Comm 2.18, Size: -200.00, Stock: 600029.SH


...

sell 平仓 601901.SH 12000
sell 平仓 600406.SH 3900
sell 平仓 601012.SH 3200
sell 平仓 600436.SH 800
sell 平仓 600346.SH 5500
sell 平仓 600025.SH 19200
sell 平仓 600519.SH -100
sell 平仓 601398.SH -35500
sell 平仓 601155.SH 2200
sell 平仓 600196.SH 3200
sell 平仓 601933.SH 10900
sell 平仓 601919.SH 15900
2019-12-16, 当前总市值 1738025.80
2019-12-17, SELL EXECUTED, Price: 6.93, Cost: 80400.00, Comm 124.74, Size: -12000.00, Stock: 601901.SH
2019-12-17, SELL EXECUTED, Price: 21.11, Cost: 80925.00, Comm 123.49, Size: -3900.00, Stock: 600406.SH
2019-12-17, SELL EXECUTED, Price: 25.55, Cost: 80576.00, Comm 122.64, Size: -3200.00, Stock: 601012.SH
2019-12-17, SELL EXECUTED, Price: 102.77, Cost: 82304.00, Comm 123.32, Size: -800.00, Stock: 600436.SH
2019-12-17, SELL EXECUTED, Price: 14.96, Cost: 82225.00, Comm 123.42, Size: -5500.00, Stock: 600346.SH
2019-12-17, SELL EXECUTED, Price: 4.37, Cost: 82368.00, Comm 125.86, Size: -19200.00, Stock: 600025.SH
2019-12-17, BUY EXECUTED, Price: 1149.70, Cost: -116300.00, Comm 57.49, 

/var/folders/6w/1xbgl6h554l61vy5w5s0zqlc0000gn/T/ipykernel_15494/2530719429.py:249: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_fac = pd.concat([df_fac,new],axis=0)
/var/folders/6w/1xbgl6h554l61vy5w5s0zqlc0000gn/T/ipykernel_15494/2530719429.py:249: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_fac = pd.concat([df_fac,new],axis=0)
/var/folders/6w/1xbgl6h554l61vy5w5s0zqlc0000gn/T/ipykernel_15494/2530719429.py:249: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA 

sell 平仓 600031.SH -5200
sell 平仓 600690.SH -4300
sell 平仓 600406.SH -3900
sell 平仓 601881.SH 7300
sell 平仓 601012.SH -3400
sell 平仓 601800.SH 9100
sell 平仓 600999.SH 4600
sell 平仓 601998.SH -13300
sell 平仓 601319.SH 10700
sell 平仓 603993.SH -22000
sell 平仓 603259.SH -800
sell 平仓 601066.SH 3000
sell 平仓 601933.SH -10800
sell 平仓 601899.SH -20500
2019-12-23, 当前总市值 1706838.59
2019-12-24, BUY EXECUTED, Price: 15.70, Cost: -81172.00, Comm 40.82, Size: 5200.00, Stock: 600031.SH
2019-12-24, BUY EXECUTED, Price: 18.95, Cost: -80840.00, Comm 40.74, Size: 4300.00, Stock: 600690.SH
2019-12-24, BUY EXECUTED, Price: 21.05, Cost: -82017.00, Comm 41.05, Size: 3900.00, Stock: 600406.SH
2019-12-24, SELL EXECUTED, Price: 11.25, Cost: 81760.00, Comm 123.19, Size: -7300.00, Stock: 601881.SH
2019-12-24, BUY EXECUTED, Price: 24.25, Cost: -81940.00, Comm 41.23, Size: 3400.00, Stock: 601012.SH
2019-12-24, SELL EXECUTED, Price: 8.75, Cost: 81991.00, Comm 119.44, Size: -9100.00, Stock: 601800.SH
2019-12-24, SELL EXECUTED, 

/var/folders/6w/1xbgl6h554l61vy5w5s0zqlc0000gn/T/ipykernel_15494/2530719429.py:249: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_fac = pd.concat([df_fac,new],axis=0)
/var/folders/6w/1xbgl6h554l61vy5w5s0zqlc0000gn/T/ipykernel_15494/2530719429.py:249: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_fac = pd.concat([df_fac,new],axis=0)
/var/folders/6w/1xbgl6h554l61vy5w5s0zqlc0000gn/T/ipykernel_15494/2530719429.py:249: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA 

ValueError: Found array with 0 sample(s) (shape=(0, 3)) while a minimum of 1 is required by RobustScaler.